# Assignment - week 3

In [31]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [32]:
tbl = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [33]:
df = pd.DataFrame(tbl[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [34]:
indexN = df[df['Borough'] == 'Not assigned'].index 
df.drop(indexN, inplace = True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [40]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [44]:
df_dropna = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_dropna.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [47]:
df_grouped = df_dropna.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(x))
df_grouped.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [49]:
na_neigh_rows = df_grouped.Neighbourhood == 'Not assigned'
df_grouped.loc[na_neigh_rows, 'Neighbourhood'] = df_grouped.loc[na_neigh_rows, 'Borough']
df_grouped[na_neigh_rows]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [61]:
df_cleaned = df_grouped
df_cleaned.shape

(103, 3)

# ---------------------

In [62]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
coors = pd.read_csv('toronto_coordinates.csv')

In [63]:
print(coors.shape)
print(coors.head())

(103, 3)
  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [64]:
df_temp = df_cleaned.set_index('Postcode')
coors_temp = coors.set_index('Postal Code')
df_coors = pd.concat([df_temp, coors_temp], axis=1, join='inner')
df_coors.index.name = 'Postcode'
df_coors.reset_index(inplace=True)
print(df_coors.shape)
df_coors.head()

(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# ------------------

In [65]:
from geopy.geocoders import Nominatim

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [67]:
map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, post, borough, neigh in zip(df_coors['Latitude'], df_coors['Longitude'], df_coors['Postcode'], df_coors['Borough'], df_coors['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
    
map

In [68]:
tor_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
tor_central_df = df_coors[toronto_df_coors['Borough'].isin(tor_boroughs)].reset_index(drop=True)
print(tor_central_df.shape)
tor_central_df.head()

(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [69]:
map = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, long, post, borough, neigh in zip(tor_central_df['Latitude'], tor_central_df['Longitude'], tor_central_df['Postcode'], tor_central_df['Borough'], tor_central_df['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
    
map

In [70]:
CLIENT_ID = '2W0LZSD1XOZERXQMTIIM5FXJ3OYPLJCLI1N3CVX1UDHXEMHR' # your Foursquare ID
CLIENT_SECRET = 'YS0QTNOHZD4VARCICVCB1TI11I3XZH30UX0MUKJFNQEZHBWA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2W0LZSD1XOZERXQMTIIM5FXJ3OYPLJCLI1N3CVX1UDHXEMHR
CLIENT_SECRET:YS0QTNOHZD4VARCICVCB1TI11I3XZH30UX0MUKJFNQEZHBWA


In [71]:
radius = 500.0
LIMIT = 100

venues = []

for lat, long, post, borough, neighbourhood in zip(tor_central_df['Latitude'], tor_central_df['Longitude'], tor_central_df['Postcode'], tor_central_df['Borough'], tor_central_df['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

venues_df = pd.DataFrame(venues)
venues_df.columns = ['Postcode', 'Borough', 'Neighbourhood', 'BorLatitude', 'BorLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1700, 9)


,Postcode,Borough,Neighbourhood,BorLatitude,BorLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,M4E,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,M4E,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,M4E,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [72]:
venues_df.groupby(['Postcode', 'Borough', 'Neighbourhood'])['VenueName'].count()

Postcode  Borough           Neighbourhood                                    
M4E       East Toronto      Business Reply Mail Processing Centre 969 Eastern      4
M4K       East Toronto      Business Reply Mail Processing Centre 969 Eastern     42
M4L       East Toronto      Business Reply Mail Processing Centre 969 Eastern     19
M4M       East Toronto      Business Reply Mail Processing Centre 969 Eastern     40
M4N       Central Toronto   Business Reply Mail Processing Centre 969 Eastern      4
M4P       Central Toronto   Business Reply Mail Processing Centre 969 Eastern      9
M4R       Central Toronto   Business Reply Mail Processing Centre 969 Eastern     21
M4S       Central Toronto   Business Reply Mail Processing Centre 969 Eastern     38
M4T       Central Toronto   Business Reply Mail Processing Centre 969 Eastern      3
M4V       Central Toronto   Business Reply Mail Processing Centre 969 Eastern     15
M4W       Downtown Toronto  Business Reply Mail Processing Centre 969 Ea

In [73]:
len(venues_df['VenueCategory'].unique())

238

In [74]:
# one hot encoding
tor_central_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
tor_central_onehot['Postcode'] = venues_df['Postcode'] 
tor_central_onehot['Borough'] = venues_df['Borough'] 
tor_central_onehot['Neighbourhoods'] = venues_df['Neighbourhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(tor_central_onehot.columns[-3:]) + list(tor_central_onehot.columns[:-3])
tor_central_onehot = tor_central_onehot[fixed_columns]

print(tor_central_onehot.shape)
tor_central_onehot.head()

(1700, 241)


,Postcode,Borough,Neighbourhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
tor_central_venues_f = tor_central_onehot.groupby(['Postcode', 'Borough', 'Neighbourhoods']).mean().reset_index()
print(tor_central_venues_f.shape)
tor_central_venues_f.head()

(38, 241)


,Postcode,Borough,Neighbourhoods,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
1,M4K,East Toronto,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.02381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02381
2,M4L,East Toronto,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
3,M4M,East Toronto,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02500
4,M4N,Central Toronto,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


In [90]:
num_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Postcode', 'Borough', 'Neighbourhoods']
freqColumns = []
for ind in np.arange(num_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = tor_central_venues_f['Postcode']
neighborhoods_venues_sorted['Borough'] = tor_central_venues_f['Borough']
neighborhoods_venues_sorted['Neighbourhoods'] = tor_central_venues_f['Neighbourhoods']

for ind in np.arange(tor_central_venues_f.shape[0]):
    row_categories = tor_central_venues_f.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
neighborhoods_venues_sorted.head()

,Postcode,Borough,Neighbourhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5V,Downtown Toronto,Business Reply Mail Processing Centre 969 Eastern,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Sculpture Garden
32,M6J,West Toronto,Business Reply Mail Processing Centre 969 Eastern,Bar,Men's Store,Asian Restaurant,Coffee Shop,Cocktail Bar,Vietnamese Restaurant,Restaurant,Bakery,Café,Pizza Place
35,M6R,West Toronto,Business Reply Mail Processing Centre 969 Eastern,Breakfast Spot,Gift Shop,Bookstore,Eastern European Restaurant,Bank,Bar,Restaurant,Italian Restaurant,Dog Run,Burger Joint
26,M5T,Downtown Toronto,Business Reply Mail Processing Centre 969 Eastern,Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Bakery,Dumpling Restaurant,Mexican Restaurant,Chinese Restaurant,Dim Sum Restaurant
25,M5S,Downtown Toronto,Business Reply Mail Processing Centre 969 Eastern,Café,Bookstore,Japanese Restaurant,Bakery,Coffee Shop,Bar,Restaurant,Jazz Club,Beer Bar,Beer Store


In [92]:
kclusters = 3

toronto_central_venues_freq_clustering = tor_central_venues_f.drop(['Postcode', 'Borough', 'Neighbourhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_central_venues_freq_clustering)

toronto_central_clustered_df = tor_central_df
toronto_central_clustered_df['Cluster'] = kmeans.labels_

toronto_central_clustered_df = toronto_central_clustered_df.join(neighborhoods_venues_sorted.drop(['Borough', 'Neighbourhoods'], 1).set_index('Postcode'), on='Postcode')
toronto_central_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
toronto_central_clustered_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,0,Playground,Gym,Restaurant,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
27,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Boutique,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Sculpture Garden
32,M6J,West Toronto,"Little Portugal,Trinity",43.647927,-79.419750,2,Bar,Men's Store,Asian Restaurant,Coffee Shop,Cocktail Bar,Vietnamese Restaurant,Restaurant,Bakery,Café,Pizza Place


In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_central_clustered_df['Latitude'], toronto_central_clustered_df['Longitude'], toronto_central_clustered_df['Postcode'], toronto_central_clustered_df['Borough'], toronto_central_clustered_df['Neighbourhood'], toronto_central_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters